In [25]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from matplotlib.pyplot import MultipleLocator
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [26]:
ori_data = pd.read_csv("q31_df_pca.csv")
data_kz = ori_data['y']
data5 = ori_data.drop(['y'], axis = 1)

In [ ]:
data5
data5 = pd.cut(data5, bins=5)

In [27]:
train = data5[:100]
label = data_kz.values[:100]
predict = data5[100:]

In [28]:
train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29
0,4.418336,4.110910,-4.571476,1.160567,-0.583769,-1.251710,-1.498994,-0.144624,-1.081969,-0.212984,...,0.939677,-0.843207,1.106732,0.778709,-0.600830,0.126878,-0.634860,-0.093491,-1.986938,1.714089
1,0.719469,-1.485016,-3.592325,-0.108989,-2.707113,0.437272,0.542298,-0.426788,0.689439,0.295181,...,-0.642566,-0.825235,-0.458695,-0.872547,0.031336,-0.058253,-0.328199,0.958966,-0.031043,-0.635396
2,5.648845,-3.339573,-1.912261,0.811085,-1.582474,-2.290038,0.647021,-0.626428,-0.439551,-0.924056,...,-0.797673,0.190045,-1.067195,-0.340648,-0.757071,0.930381,0.863314,-0.625970,0.183925,0.359224
3,3.365403,-4.066140,5.507794,-3.942182,0.025627,-0.881894,0.116458,2.341291,2.002114,3.626196,...,0.287773,-1.325969,0.350234,-0.611359,0.092804,1.568779,-0.109939,0.111520,0.456633,0.904662
4,3.849019,0.501642,-2.421548,-2.223887,2.217132,0.618889,-2.923348,-3.836563,1.950592,-0.320196,...,-2.190668,-1.395366,2.037302,-0.220556,0.346945,0.477080,-0.553159,0.782542,-0.736013,0.945874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-3.821476,-0.525164,0.004065,-0.485820,0.626990,0.211552,-1.604908,0.400305,-2.305952,0.870405,...,0.781630,-1.231337,0.009634,0.858026,1.682709,-0.263049,-0.638621,-0.585413,-0.482750,-0.108231
96,-4.023014,3.836824,0.100828,-1.022225,-2.189299,-0.453024,-0.557269,0.004657,-1.413390,0.047422,...,-0.447533,-0.031419,-0.899499,-0.846414,-0.218920,-0.312357,0.525290,-0.340002,0.136308,-0.035799
97,13.615124,-1.500915,-4.049243,2.338023,-1.565479,-5.827916,4.021817,1.514435,-1.832585,0.576140,...,-0.446530,-0.219320,0.885727,-2.446942,-1.206819,0.190684,-0.366743,-0.171342,-1.609954,-0.861398
98,0.799481,-3.087444,0.556230,0.788304,-1.337726,0.479187,0.805182,3.919591,1.582935,-1.750936,...,0.029370,-0.079187,-1.454667,1.430652,0.600367,-0.244955,-1.528633,-1.697742,-0.263115,1.117791


In [29]:
sm = SMOTE(random_state=0)
xres, yres = sm.fit_resample(train.values, label)
xtrain, xvalid, ytrain, yvalid = train_test_split(train.values,label,random_state=50,test_size=0.2)
xtrain_res, xvalid_res, ytrain_res, yvalid_res = train_test_split(xres,yres,random_state=50,test_size=0.2)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 4, n_neighbors = 6

In [30]:
# 定义参数网格
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_xgb = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'max_depth': [5, 10, 20, 30, 40],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

param_grid_mlp = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100)],
    'activation': ['relu', 'tanh'],
}

param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'], 
}

In [31]:
# 采样前
def function_before(model, param_grid, cv_values=[5], n_jobs=16):
    best_auc = 0
    best_cv = None
    best_model = None
    best_params = None

    for cv in cv_values:
        grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=n_jobs)
        grid_search.fit(train.values, label)

        if grid_search.best_score_ > best_auc:
            best_model = grid_search.best_estimator_
            best_auc = grid_search.best_score_
            best_params = grid_search.best_params_
            best_cv = cv

    if best_model is None:
        raise ValueError("No model was trained successfully.")

    acc = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='accuracy').mean(), 2)
    recall = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='recall').mean(), 2)
    precision = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='precision').mean(), 2)
    f1 = round(cross_val_score(best_model, train.values, label, cv=best_cv, scoring='f1').mean(), 2)

    result_dict = {
        'best_model': best_model,
        'best_auc': round(best_auc, 2),
        'best_params': best_params,
        'acc': acc,
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'best_cv': best_cv
    }

    return result_dict

cv_values = [5, 10]
best_results_rf = function_before(RandomForestClassifier(random_state=100), param_grid_rf, cv_values)
best_results_lgbm = function_before(LGBMClassifier(random_state=100), param_grid_lgbm, cv_values)
best_results_xgb = function_before(XGBClassifier(random_state=100), param_grid_xgb, cv_values)
best_results_svc = function_before(SVC(probability=True, random_state=100), param_grid_svc, cv_values)
best_results_mlp = function_before(MLPClassifier(random_state=100), param_grid_mlp, cv_values)
best_results_lr = function_before(LogisticRegression(random_state=100), param_grid_lr, cv_values)

model0 = best_results_rf['best_model']
model1 = best_results_lgbm['best_model']
model2 = best_results_xgb['best_model']
model3 = best_results_svc['best_model']
model4 = best_results_mlp['best_model']
model5 = best_results_lr['best_model']

model0.fit(xtrain,ytrain)
model1.fit(xtrain,ytrain)
model2.fit(xtrain,ytrain)
model3.fit(xtrain,ytrain)
model4.fit(xtrain,ytrain)
model5.fit(xtrain,ytrain)

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detect

KeyboardInterrupt: 

In [ ]:

def f(model_list,name_list,types='train'):
    plt.figure(figsize=(8, 7), dpi=100, facecolor='w')    # dpi:每英寸长度的像素点数；facecolor 背景颜色
    plt.xlim((-0.01, 1.02))  # x,y 轴刻度的范围
    plt.ylim((-0.01, 1.02))
    plt.xticks(np.arange(0, 1.1, 0.1))  #绘制刻度
    plt.yticks(np.arange(0, 1.1, 0.1))
    
    if types == 'test':
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xvalid)[:,1]
            fpr, tpr, _ = metrics.roc_curve(yvalid, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2.5, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    else:
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xtrain)[:,1]
            fpr, tpr, _ = metrics.roc_curve(ytrain, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2.5, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    plt.legend(loc='best',fontsize=15)    # 设置显示标签的位置
    plt.xlabel('假阳率', fontsize=14)   #绘制x,y 坐标轴对应的标签
    plt.ylabel('真阳率', fontsize=14)
    plt.tick_params(labelsize=23)

    plt.grid(b=True, ls=':')  # 绘制网格作为底板;b是否显示网格线；ls表示line style
    plt.savefig(f'q1_roc_auc({types}(采样前)).png',dpi=1000)
    plt.show()

f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'test')
f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'train')

In [ ]:
# 采样后
def function_after(model, param_grid, cv_values=[5], n_jobs=16):
    best_auc = 0
    best_cv = None
    best_model = None
    best_params = None

    for cv in cv_values:
        grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=n_jobs)
        grid_search.fit(xres, yres)
        
        if grid_search.best_score_ > best_auc:
            best_model = grid_search.best_estimator_
            best_auc = grid_search.best_score_
            best_params = grid_search.best_params_
            best_cv = cv

    if best_model is None:
        raise ValueError("No model was trained successfully.")

    acc = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='accuracy').mean(), 2)
    recall = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='recall').mean(), 2)
    precision = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='precision').mean(), 2)
    f1 = round(cross_val_score(best_model, xres, yres, cv=best_cv, scoring='f1').mean(), 2)
    
    result_dict = {
        'best_model': best_model,
        'best_auc': round(best_auc, 2),
        'best_params': best_params,
        'acc': acc,
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'best_cv': best_cv
    }
    
    return result_dict

cv_values = [5, 10]
best_results_rf = function_after(RandomForestClassifier(random_state=100), param_grid_rf, cv_values)
best_results_lgbm = function_after(LGBMClassifier(random_state=100), param_grid_lgbm, cv_values)
best_results_xgb = function_after(XGBClassifier(random_state=100), param_grid_xgb, cv_values)
best_results_svc = function_after(SVC(probability=True, random_state=100), param_grid_svc, cv_values)
best_results_mlp = function_after(MLPClassifier(random_state=100), param_grid_mlp, cv_values)
best_results_lr = function_after(LogisticRegression(random_state=100), param_grid_lr, cv_values)

model0 = best_results_rf['best_model']
model1 = best_results_lgbm['best_model']
model2 = best_results_xgb['best_model']
model3 = best_results_svc['best_model']
model4 = best_results_mlp['best_model']
model5 = best_results_lr['best_model']

model0.fit(xtrain_res,ytrain_res)
model1.fit(xtrain_res,ytrain_res)
model2.fit(xtrain_res,ytrain_res)
model3.fit(xtrain_res,ytrain_res)
model4.fit(xtrain_res,ytrain_res)
model5.fit(xtrain_res,ytrain_res)

In [ ]:
def f(model_list,name_list,types='train'):
    plt.figure(figsize=(8, 7), dpi=100, facecolor='w')    # dpi:每英寸长度的像素点数；facecolor 背景颜色
    plt.xlim((-0.01, 1.02))  # x,y 轴刻度的范围
    plt.ylim((-0.01, 1.02))
    plt.xticks(np.arange(0, 1.1, 0.1))  #绘制刻度
    plt.yticks(np.arange(0, 1.1, 0.1))
    
    if types == 'test':
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xvalid_res)[:,1]
            fpr, tpr, _ = metrics.roc_curve(yvalid_res, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2.5, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    else:
        for model,name in zip(model_list,name_list):
            ytest_prob = model.predict_proba(xtrain_res)[:,1]
            fpr, tpr, _ = metrics.roc_curve(ytrain_res, ytest_prob)
            auc = metrics.auc(fpr, tpr)
            plt.plot(fpr, tpr, '-', lw=2.5, label=f'{name} AUC:%.5f' % auc)  # 绘制AUC 曲线
    plt.legend(loc='best',fontsize=15)    # 设置显示标签的位置
    plt.xlabel('假阳率', fontsize=14)   #绘制x,y 坐标轴对应的标签
    plt.ylabel('真阳率', fontsize=14)
    plt.tick_params(labelsize=23)

    plt.grid(b=True, ls=':')  # 绘制网格作为底板;b是否显示网格线；ls表示line style
    plt.savefig(f'q1_roc_roc_auc({types}（采样后）).png',dpi=1000)
    plt.show()

f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'test')
f([model0, model1, model2, model3, model4, model5], ['RF', 'LGBM', 'XGB', 'SVC', 'MLP', 'LR'], 'train')

In [ ]:
best_results_mlp

In [ ]:
best_results_svc

In [ ]:
mlp_model = best_results_mlp['best_model']
mlp_model.fit(xres,yres)
mlp_model_result = [round(i,4) for i in mlp_model.predict_proba(data5.values)[:,1]]
mlp_model_result = pd.DataFrame({'扩张概率':mlp_model_result})
mlp_model_result.to_excel('q12扩张概率.xlsx',index=False)

In [ ]:
import pickle
# 保存最优的参数
mlp_model = best_results_mlp['best_model']
with open('mlp_default_params.pkl', 'wb') as f:
    pickle.dump(mlp_model.get_params(), f)

In [ ]:
import pickle
# 保存最优的参数
xgb_model = best_results_xgb['best_model']
with open('xgboost_default_params.pkl', 'wb') as f:
    pickle.dump(xgb_model.get_params(), f)

# # # 调用已保存的函数参数
# # with open('xgboost_default_params.pkl', 'rb') as f:
# #     default_params = pickle.load(f)
# # xgb_model = xgb.XGBClassifier(**default_params)
# xgb_model.fit(xres,yres)
# xgb_model_result = [round(i,4) for i in xgb_model.predict_proba(data5.values)[:,1]]

In [ ]:
# xgb_model_result = pd.DataFrame({'扩张概率':xgb_model_result})
# xgb_model_result.to_excel('q12扩张概率.xlsx',index=False)

In [ ]:
mlp_model.fit(xres,yres)
mlp_model_result = [round(i,4) for i in mlp_model.predict_proba(data5.values)[:,1]]
mlp_model_result = pd.DataFrame({'扩张概率':mlp_model_result})
mlp_model_result.to_excel('q12扩张概率.xlsx',index=False)
# 'name':data1['ID'].values,
mlp_model_result